In [1]:
from ipywidgets import FileUpload, Output
import ipywidgets as widgets
from IPython.display import display, Javascript,  HTML
import sys
from io import StringIO
import pandas as pd
import plotly.express as px
import numpy as np 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.signal import find_peaks

out = Output()
upload = FileUpload(accept='.txt', multiple=True)

upload 

FileUpload(value={}, accept='.txt', description='Upload', multiple=True)

In [2]:
surfaceInput = widgets.BoundedFloatText(
    value=0.27,
    min=0,
    max=10.0,
    step=0.1,
    description='Surface:',
    disabled=False
)
display(surfaceInput)

outputfilename = widgets.Text(
    value='',
    placeholder='Type the output file name',
    description='String:',
    disabled=False
)
display(outputfilename)

BoundedFloatText(value=0.27, description='Surface:', max=10.0, step=0.1)

Text(value='', description='String:', placeholder='Type the output file name')

In [3]:
# name = input('Please upload a file and a surface value. Press Enter when done.  \n')
def on_button_clicked(b):
    with out:
        out.clear_output()
        data = []
        if(bool(upload.value) and outputfilename.value!=''):  
            for uploaded_filename in upload.value:
                content = upload.value[uploaded_filename]['content']   
                print(uploaded_filename)

                s = "" 
                for line in content.split(b'\n'):
                    if "Source;Delais(s);Mesure ;Temps(s);Mesure/Source;Source/Mesure;Puissance" in line.decode('latin-1'):
                        s = ""
                    s +=line.decode('latin-1')

                df = pd.read_csv(StringIO(s), sep=";")
                surface = surfaceInput.value
                df['J'] = df['Mesure ']*1000/surface
                df['Power'] = df['J']*df['Source']

                fig = make_subplots(rows=1, cols=2)

                fig1 = px.line(df,x="Source", y="J")
                fig2 = px.line(df,x="Source", y="Power")

                fig.add_trace(fig1['data'][0], row=1, col=1)
                fig.add_trace(fig2['data'][0], row=1, col=2)

                fig.add_trace(go.Scatter(x=[0],y=[np.interp(0, df.Source, df.J)],mode='markers'),row =1 , col =1)
                fig.add_trace(go.Scatter(x=[np.interp(0, df.J, df.Source)],y=[0],mode='markers'),row =1 , col =1)
                minPower = min(df.Power)
                fig.add_trace(go.Scatter(x=[np.array(df[df.Power == min(df.Power)].Source)[0]],y=[minPower],mode='markers'),row =1 , col =2) 
                
                fig.update_layout(
                    showlegend=False,
                    autosize=False,
                    width=1500,
                    height=700,
                    margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=3
                    )
                )
                # Update xaxis properties
                fig.update_xaxes(title_text="Voltage [V]", row=1, col=1)
                fig.update_xaxes(title_text="Voltage [V]", row=1, col=2)

                # Update yaxis properties
                fig.update_yaxes(title_text="J [mA/cm^2] ", row=1, col=1)
                fig.update_yaxes(title_text="Power [mW/cm^2]", row=1, col=2)

                fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')
                fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')

                # Update title and height
                fig.update_layout(title_text=uploaded_filename)
                fig.show()
                
                Jsc = round(np.abs(np.interp(0, df.Source, df.J)),3)
                Voc = round(np.interp(0, df.J, df.Source),3)
                Jmax = round(np.abs(np.array(df[df.Power == min(df.Power)].J)[0]),3)
                Vmax = round(np.array(df[df.Power == min(df.Power)].Source)[0],3);
                FF = round(((Vmax*Jmax)/(Voc*Jsc))*100,3)
                PCE = round((Voc*Jsc*FF)/100,3)
                
                print("Jsc is: " + str(Jsc))
                print("Voc is: " + str(Voc))
                print("Minimum power: " + str(min(df.Power)) + " at " + str(Vmax) + " V and " + str(Jmax) + " mA/cm^2."  )
                
                data.append([uploaded_filename,Voc,Jsc,Vmax,Jmax,round(Voc*Jsc,3),round(Vmax*Jmax,3),FF,PCE,surface])
            upload.value.clear()
            upload._counter = 0
            dftable = pd.DataFrame(data, columns=['sample', 'Voc (V)','Jsc (mA/cm2)','Vmax','Jmax','Voc*Jsc','Vmax*Jmax','FF(%)','PCE (%)','surface'])
            display(dftable)
            dftable.to_csv('out_'+outputfilename.value+'.csv', index=False)
        else: 
            print("Empty file")

In [4]:
button = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check', # (FontAwesome names without the `fa-` prefix)
)
button.on_click(on_button_clicked)
display(button)
out

Button(description='Plot', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()